# Assignment 1 Comp 521 Fall 2015

Assigned: 3 September 2015  
Due: 17 September 2015

Answer each question below by filling in code at the point indicated. Do not modify any of the other cells. 

You __must__ run the cells in order. It looks like you can come back later after restarting and pick up where you left off but that is not the case. When restarting you always have to start at the top of the notebook and run each cell in turn. 

In [27]:
# fill in the information below
Author = 'tlwu'
Collaborators = []

In [28]:
# checker setup do NOT change this
import comp521
check, report = comp521.start('A1')

In [29]:
# setup, no need to change this
from Dee import *
from DeeDatabase import Database
from CourseEnrollmentDee import DeeDB
import sqlite3

# I'll open a DB in memory
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

## Question 1
Use SQL to create tables for the following database schema modeling a library.

STUDENT __sid__(INTEGER) sname(TEXT)  
BOOK __bid__(INTEGER) bname(TEXT) author(TEXT)  
BORROWS __id__(INTEGER) sid(INTEGER) bid(INTEGER) date(TEXT)  

All primary keys are bold. In the BORROWS table, __bid__ is a foreign key referencing BOOK and __sid__ is a foreign key referencing STUDENT.

__Make sure__ you use the *exact* names and data types for the tables and attributes in order to pass the automatic test.

In [30]:
# I'll clean up for you so you don't get tripped up by trying more than once. Leave this alone
cursor.execute('DROP TABLE IF EXISTS "STUDENT"')
cursor.execute('DROP TABLE IF EXISTS "BOOK"')
cursor.execute('DROP TABLE IF EXISTS "BORROWS"')

# insert your code to create the table here using cursor to create the tables.
cursor.execute("""
    create table STUDENT
        (sid integer primary key,
         sname text
        )
""")

cursor.execute("""
    create table BOOK
        (bid integer primary key,
         bname text,
         author text
        )
""")

cursor.execute("""
    create table BORROWS
        (id integer,
         sid integer,
         bid integer,
         date text,
         primary key (id)
         foreign key (sid) references STUDENT
         foreign key (bid) references BOOK)
""")

# leave this testing code alone
c1 = cursor.execute("SELECT name from sqlite_master WHERE type='table' order by name asc").fetchall()
check('Q1a', c1, points=6)

Q1a correct


## Question 2
Insert the following data into the tables you created above.

<table>
<caption>STUDENT</caption>
<tr><th>sid</th><th>sname</th></tr>
<tr><td>1</td><td>Alice</td></tr>
<tr><td>2</td><td>Bob</td></tr>
<tr><td>3</td><td>Cindy</td></tr>
</table>

<table>
<caption>BOOK</caption>
<tr><th>bid</th><th>bname</th><th>author</th></tr>
<tr><td>1</td><td>Database Management Systems</td><td>Ramakrishnan</td></tr>
<tr><td>2</td><td>Lord of the Rings</td><td>Tolkien</td></tr>
</table>

<table>
<caption>BORROWS</caption>
<tr><th>id</th><th>sid</th><th>bid</th><th>date</th></tr>
<tr><td>1</td><td>2</td><td>1</td><td>8/3/2015</td></tr>
<tr><td>2</td><td>3</td><td>2</td><td>9/1/2015</td></tr>
</table>


In [31]:
# write your code here
student = [(1,"Alice"),
           (2,"Bob"),
           (3,"Cindy"),]
for student in student:
    cursor.execute("insert into STUDENT values (?,?)", student)
    
book = [(1,"Database Management Systems","Ramakrishnan"),
        (2,"Lord of the Rings","Tolkien")]
for book in book:
    cursor.execute("insert into BOOK values (?,?,?)", book)
    
borrows = [(1,2,1,"8/3/2015"),
           (2,3,2,"9/1/2015")]
for borrow in borrows:
    cursor.execute("insert into BORROWS values (?,?,?,?)", borrow)
           
# testing code
c2a = cursor.execute('select * from STUDENT').fetchall()
check('Q2a', c2a, points=3)
c2b = cursor.execute('select * from BOOK').fetchall()
check('Q2b', c2b, points=3)
c2c = cursor.execute('select * from BORROWS').fetchall()
check('Q2c', c2c, points=3)

Q2a correct
Q2b correct
Q2c correct


## Question 3
Delete the record with id equal 1 from the BORROWS table.

In [32]:
# write your code here
cursor.execute("""
    delete 
    from BORROWS 
    where id = 1
""")
# don't touch this
c3 = cursor.execute('''select * from BORROWS''').fetchall()
check('Q3', c3, points=5)

Q3 correct


## Setup for questions 4-12
Consider the following schema for a database modeling courses at UNC.
```
    DEPARTMENT (did, name)  
    COURSE (cid, did, name, num, creditHours)  
    STUDENT (sid, fname, lname, did)  
    ENROLLED_IN (eid, sid, cid)  
```
    
In the COURSE table, did is a foreign key referencing DEPARTMENT(did). In the ENROLLED_IN
table, sid is a foreign key referencing STUDENT(sid), and cid is a foreign key referencing
COURSE(cid). In STUDENT, did is a foreign key referencing DEPARTMENT(did).

### Questions 4-8

Use [Dee](http://www.quicksort.co.uk/DeeDoc.html) to write relational algebra queries for the following. In particular, you may find the
following Dee operators most useful: Projection, Restriction (corresponds to select in
relational algebra), Join, Intersection, Difference, Union, and Divide.

Your, already populated, Dee database is named DeeDB.


## Question 4
Find the names of all courses below the 500 level. You can get the Dee relation for COURSE with `DeeDB.COURSE`.

In [33]:
# write code here used DeeDB and Dee methods.
courseNames = DeeDB.COURSE.where(lambda t : t.num<500)
c4 = courseNames.project(['name'])


# checking code
check('Q4', c4, points=5)

Q4 correct


## Question 5
Find the first and last names of all students enrolled in "Files and Databases".

In [34]:
# Assign your result to c5

class521 = DeeDB.COURSE.where(lambda t : t.name=="Files and Databases")
enrolled = class521 & DeeDB.ENROLLED_IN
enrolled_students = enrolled & DeeDB.STUDENT.project(['sid','lname','fname'])
c5 = enrolled_students.project(['lname','fname'])

# checking
check('Q5', c5, points=5)

Q5 correct


## Question 6
Find the first and last names of all students in the ANTH department.

In [35]:

# assign to c6
anthDep = DeeDB.DEPARTMENT.where(lambda t : t.name=='ANTH')
anthStudents = DeeDB.DEPARTMENT & DeeDB.STUDENT & anthDep
c6 = anthStudents.project(['lname','fname'])
# checking
check('Q6', c6, points=5)

Q6 correct


## Question 7
List the first and last names of all students enrolled in no courses.

In [36]:
# assign to c7
enrolled_students = DeeDB.STUDENT & DeeDB.ENROLLED_IN.project(['sid'])
c7 = DeeDB.STUDENT.project(['lname','fname']) - enrolled_students.project(['lname','fname'])

# checking
check('Q7', c7, points=5)

Q7 correct


## Question 8
List the first and last names of all students enrolled in every COMP course.

In [37]:
compDep = DeeDB.DEPARTMENT.where(lambda t : t.name=='COMP')
compCourses = compDep.project(['did']) & DeeDB.COURSE
var1 = (compCourses & DeeDB.ENROLLED_IN) & DeeDB.STUDENT.project(['lname','fname','sid'])
c8 = DIVIDE_SIMPLE( var1.project(['cid','lname','fname']),compCourses.project(['cid']))





check('Q8', c8, points=15)

Q8 correct


### Questions 9-12

Now use the sqlite3 database `courseEnrollment.db`, which has the same schema as above to write SQL queries for the following. I will open it for you below. __You must execute this next cell before the others!__

In [38]:
econn = sqlite3.connect('CourseEnrollment.db')
ecursor = econn.cursor()

# use ecursor to refer to this database below.

## Question 9
Find the names of all 4 credit hour courses.

In [49]:
# your code here
a = ecursor.execute("""
select distinct S.fname
    from Student S
    where S.fname LIKE '%b%'
""")
    
print [item[0] for item in a]

# c9 = ecursor.execute("""
# select C.name
#     from COURSE C
#     where C.creditHours == 4
# """).fetchall()
    
# check('Q9', c9, points=10)

[u'BARBARA', u'ROBERT']


## Question 10
Find the first and last names of all students enrolled in COMP courses.

In [42]:
# your code here


c10 = ecursor.execute("""
select distinct S.fname, S.lname
    from COURSE C, DEPARTMENT D, STUDENT S, ENROLLED_IN E
    where D.name=="COMP" and C.did == D.did and S.sid == E.sid and E.cid == C.cid
""").fetchall()
    

check('Q10', c10, points=10)

Q10 correct


## Question 11
Find the first and last names of all students enrolled in courses of at least 500 level.

In [43]:
# your code here
c11 = ecursor.execute("""
select distinct S.fname, S.lname
    from STUDENT S, ENROLLED_IN E, COURSE C
    where C.num >= 500 and S.sid == E.sid and E.cid == C.cid
""").fetchall()


check('Q11', c11, points=10)

Q11 correct


## Question 12
Find the first and last names of all students enrolled in 'Data Structures' and "Anthropology of the Body and the Subject'.

In [78]:
# your code here
c12 = ecursor.execute("""
select distinct S.fname, S.lname
    from STUDENT S, ENROLLED_IN E, COURSE C
    where C.name == 'Data Structures' and S.sid == E.sid and C.cid == E.cid
    
intersect

select distinct S.fname, S.lname
    from STUDENT S, ENROLLED_IN E, COURSE C
    where C.name == 'Anthropology of the Body and the Subject' and S.sid == E.sid and C.cid == E.cid 
""").fetchall()


check('Q12', c12, points=15)

Q12 correct


## Done!

Now get your report and submit your assignment.

In [79]:
report(Author, Collaborators)

Q10
Q11
Q12
Q1a
Q2a
Q2b
Q2c
Q3
Q4
Q5
Q6
Q7
Q8
Q9
Report for tlwu
  Collaborators: []
  14 of 14 correct, 100 of 100 points
